In [190]:
import pandas as pd
from Crypto.Cipher import AES
import numpy as np
import io
import os
from PIL import Image
from Crypto.Hash import CMAC


In [191]:
filename="output.csv"

out=pd.read_csv(filename)
size=267
addr=0
enc=np.empty(size,dtype=object)
out = out[pd.to_numeric(out['cntr'], errors='coerce').notna()]
out = out[pd.to_numeric(out['done'], errors='coerce').notna()]
out['cntr'] = out['cntr'].astype(int)
out['done'] = out['done'].astype(int)
out['encrypted128'] = out['encrypted128'].astype(str)

out['messAddra']=out['messAddra'].astype(int)
tag=0
out

,cntr,size,messAddra,cmacAddra,messIn,cmacIn,encrypted128,done,tag
1,0,34176,0,0,xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx,xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx,xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx,0,xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx
2,1,34176,1,0,ffd8ffe000104a464946000101000001,ffd8ffe000104a464946000101000001,xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx,0,xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx
3,2,34176,1,0,00010000ffdb00430008060607060508,ffd8ffe000104a464946000101000001,xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx,0,xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx
4,3,34176,2,0,00010000ffdb00430008060607060508,ffd8ffe000104a464946000101000001,xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx,0,xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx
5,4,34176,2,0,0707070909080a0c140d0c0b0b0c1912,ffd8ffe000104a464946000101000001,xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx,0,xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx
...,...,...,...,...,...,...,...,...,...
24994,18,34176,267,267,xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx,xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx,f684d0e24a2670f3fe516bce0f41437a,1,7ce783094b8a7710957814628a2c6bdf
24995,19,34176,267,267,xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx,xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx,f684d0e24a2670f3fe516bce0f41437a,1,7ce783094b8a7710957814628a2c6bdf
24996,20,34176,267,267,xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx,xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx,f684d0e24a2670f3fe516bce0f41437a,1,7ce783094b8a7710957814628a2c6bdf
24997,21,34176,267,267,xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx,xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx,f684d0e24a2670f3fe516bce0f41437a,1,7ce783094b8a7710957814628a2c6bdf


In [192]:
def decrypt_data(encrypted_hex, key):
    try:
        # Convert the encrypted text from hex to bytes
        encrypted_data = bytes.fromhex(encrypted_hex)
        
        if len(encrypted_data) != 16:
            raise ValueError("Encrypted data must be 128 bits (16 bytes)")

        # Create a new AES cipher in ECB mode
        cipher = AES.new(key, AES.MODE_ECB)
        
        # Decrypt the data
        decrypted_data = cipher.decrypt(encrypted_data)        
        # Return decrypted data in hexadecimal format
        return decrypted_data.hex()
    except ValueError as e:
        return f"Error: {e}"

In [193]:
key = bytes.fromhex('000102030405060708090a0b0c0d0e0f')

for i in range(out.shape[0]):
    row=out.iloc[i]
    
    if(row["encrypted128"].strip()!="xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx"):
        if(addr<size):
            if(row["cntr"]==44 or row["cntr"]==46 or row["cntr"]==2):
                enc[addr]=row["encrypted128"]
                addr=addr+1
                # print(row["encrypted128"])
        if(row["done"]==1):
            tag=row["tag"]


In [194]:
print(enc)

[' 9958fdec8e2d65c80160cfc0e66432fa' ' defc899e94c6b8ce8fedcee9e9ff3dcf'
 ' e109ac33e298773a8a915b24c78e054f' ' 5d8b8af9f919c8e6dfc310d44dc36c19'
 ' 9292c41266e1f4e3e689ec80dfb602fe' ' 8e9bb2290016cf2ac8435a4b637f6960'
 ' 2f48d8f5931762ea6b0bf3e744aac4ae' ' bac8daba053b033e9cb6ddae9c65a934'
 ' bac8daba053b033e9cb6ddae9c65a934' ' d7d923066a3838893bf8d6fffb3f3aa9'
 ' 09ddf7706a142ac33230359552c37a0e' ' 05e30439f794d9a3151fcc2eb635237f'
 ' 611f51ea571ee404881681395764c850' ' 18e5c4a6a0c54277910556af665d45bd'
 ' a7e088c2229dd592a544e5c7fd04006b' ' 5e33e0030daee23d841d44eec6dbf1cb'
 ' 2550e50dd872bd5e970dd5ec159488d1' ' c695c098a89435f1d967f5fd9bca0f72'
 ' 57f949ca22e7df1804b4e314235960a9' ' 573d6e9660f81c9b59bb283fae9dd382'
 ' f6161fd7e65e8138b5333873ee8435ba' ' fd9aa2374613d4412c91f29846c72dbc'
 ' 2f4c6db121b235de7dcaa0f463ae26fc' ' b3e9845de519ebe1a2c2f7e62027ec27'
 ' a1bd127c01f0238297a11a26e6ee8677' ' 5c4942767bce9e6c03173e06c44add64'
 ' dd07ea46b4ca112ad96534631afc8226' ' def59acc72fb

In [195]:
decrypted_array = np.array([decrypt_data(x, key) for x in enc])
decrypted_array

array(['ffd8ffe000104a464946000101000001',
       '00010000ffdb00430008060607060508',
       '0707070909080a0c140d0c0b0b0c1912',
       '130f141d1a1f1e1d1a1c1c20242e2720',
       '222c231c1c2837292c30313434341f27',
       '393d38323c2e333432ffdb0043010909',
       '090c0b0c180d0d1832211c2132323232',
       '32323232323232323232323232323232',
       '32323232323232323232323232323232',
       '3232323232323232323232323232ffc0',
       '00110800800080030122000211010311',
       '01ffc4001f0000010501010101010100',
       '00000000000000010203040506070809',
       '0a0bffc400b510000201030302040305',
       '0504040000017d010203000411051221',
       '31410613516107227114328191a10823',
       '42b1c11552d1f02433627282090a1617',
       '18191a25262728292a3435363738393a',
       '434445464748494a535455565758595a',
       '636465666768696a737475767778797a',
       '838485868788898a9293949596979899',
       '9aa2a3a4a5a6a7a8a9aab2b3b4b5b6b7',
       'b8b9bac2c3c4c5c6c7c8c9cad2d3d4d5',
       'd6d

In [196]:
def compute_cmac(plaintexts_hex, key_hex):
    cobj = CMAC.new(key, ciphermod=AES)
    
    # Convert hex strings in the NumPy array to bytes and update CMAC
    for hex_str in plaintexts_hex:
        plaintext = bytes.fromhex(hex_str)  # Convert hex string to bytes
        cobj.update(plaintext)
    
    # Print final CMAC
    print("CMAC:", cobj.hexdigest())
    return cobj.hexdigest()

In [197]:
cmac=compute_cmac(decrypted_array, key)

if(cmac.strip()==tag.strip()):
    print("Authentication Successful!")
    # print(out.iloc[-1]["tag"])
else:
    print(tag)
    print("Authentication Failed!") 

CMAC: 7ce783094b8a7710957814628a2c6bdf
Authentication Successful!


In [198]:


original_bin_file_path = "encoded_image.bin"

# Get the number of bits in the original file
original_file_size = os.path.getsize(original_bin_file_path)  # In bytes
original_file_bits = original_file_size * 8

with open("decrypted_image.bin", "wb") as bin_file:
    for hex_string in decrypted_array:
        # Only process valid hex strings
        if hex_string and "Error" not in hex_string:
            # Convert hex string to binary data (bytes)
            binary_data = bytes.fromhex(hex_string)
            binary_data_bits = len(binary_data) * 8  # Convert length to bits
            
            # Truncate extra bits if necessary
            if binary_data_bits > original_file_bits:
                # Calculate the number of bytes to truncate
                truncate_bytes = (binary_data_bits - original_file_bits) // 8
                binary_data = binary_data[:-truncate_bytes]  # Truncate the extra bytes

            # Write the truncated (or unchanged) binary data to the file
            bin_file.write(binary_data)

            # After writing, reduce the remaining required bits
            original_file_bits -= len(binary_data) * 8
            
            if original_file_bits <= 0:
                break  

In [199]:
bin_file_path = "decrypted_image.bin"
jpeg_file_path = "output_image.jpeg"

# Read the .bin file and write it directly to a .jpeg file
with open(bin_file_path, "rb") as bin_file:
    bin_data = bin_file.read()

# Write the binary data to a .jpeg file
with open(jpeg_file_path, "wb") as jpeg_file:
    jpeg_file.write(bin_data)

image = Image.open(jpeg_file_path)

image.show()